## Step0 確認日期

In [6]:
import datetime
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
yesterday = yesterday.strftime("%Y%m%d")  
#print yesterday

20160520


## Step1 存取昨天新聞的URL

In [8]:
#把分類跟 url 組合在一起
import requests
from bs4 import BeautifulSoup
from urlparse import urljoin
import os
import datetime as dt
import time
#製造日期
import datetime
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
yesterday = yesterday.strftime("%Y%m%d")  
urls = ""
#印出每一天的蘋果新聞 url
page_url = 'http://www.appledaily.com.tw/appledaily/archive/{}'.format(yesterday)
#print page_url  
main_url ='http://www.appledaily.com.tw/appledaily'
res = requests.get(page_url)
soup = BeautifulSoup(res.text)
#包含所有種類的div
div = soup.select('.abdominis.clearmen')[0]
#抓取所有連結
#news_url=[]
article = div.select('.nclns')

#兩層迴圈，把類別跟 url 組合起來
for i in article:
    f = open('C:/Users/BIG DATA/Desktop/Apple_news/{}.text'.format(yesterday),'w')
    category = i.select('h2')[0].text.encode("utf8")  
    for li in i.select('li'):
        href = li.select('a')[0]['href']
        news_url = urljoin(main_url,href).encode("utf8") +'\n' #urljoin會自動合併兩個網址
        urls += category + news_url  #先都存起來
time.sleep(0.6)
f.write(urls) #用encode('utf-8)    #最後再一次寫出來
f.close()    

http://www.appledaily.com.tw/appledaily/archive/20160520


## Step2 讀取昨天的URL檔案，並將內容抓下來

In [13]:
#這個才是最正確的
#修改成一次爬好幾個月
import string
import datetime as dt
import requests
import time
from bs4 import BeautifulSoup
import json
import time
count = 0
#製造日期的迴圈
all_json_str = []
f = open('C:/Users/BIG DATA/Desktop/Apple_news/{}.text'.format(yesterday),'r')
for url_no in f.readlines():
    try:
        category = url_no.split('http')[0]        #把類別抓出來
        url = 'http'+url_no.split('http')[1]      #重新組合url
        res = requests.get(url)
        soup = BeautifulSoup(res.text)
        file_date,file_num = url.split('/')[6],url.split('/')[7]  #替檔案取名子    

        #抓標題跟副標題
        title1 = soup.select('#h1')[0].text
        title2 = soup.select('#h2')[0].text            
        title =''
        titles = str(title1.encode('utf-8'))+str(title2.encode('utf-8'))
        title = ','.join(titles.split())
        #print title

        #抓內文
        article = ''
        div= soup.select('.articulum')[0]   #先把所有內文抓到
        ps = div.select('p')                #再把裡面所有的p標籤抓到
        for p in ps:                        #把每個p標籤一個一個抓出來印
            article += p.text.replace('\r','')  #段落會隱藏換行符號\r，要取代掉，不然後面讀取json格式會只讀到最後一行

        #抓日期
        #date = soup.select('.gggs')[0].select('time')[0].text
        #f.write(time.encode('utf-8')+'\n')

        data  =  { 
            'comp':'apple',  #新聞來源
            'category':category,
            'title':title, 
            'content':article,
            'date':file_date,  #直接抓檔案的日期來用
            'url':url,
            'keyw':'',
            'hitcount':0
        }

        all_json_str.append(data) #加個,把多本字典分開來
        #爬完一個網址休息一下
        time.sleep(1)
    #印出有問題的新聞
    except IndexError:
        count +=1
        #print count
        continue
#url 檔案關閉
f1 = open('C:/Users/BIG DATA/Desktop/Apple_news/json/{}.json'.format(yesterday),'w')
b =json.dumps(all_json_str) #再把 list 字典轉換成 json 並包起來
f1.write(b)
f1.close()
f.close()

1
2
3
4
5
6
7
8
9
10
11
12


## Step3 讀取檔案並將新聞存入MongoDB

In [14]:
import pymongo
from pymongo import MongoClient
import json

with open('C:/Users/BIG DATA/Desktop/Apple_news/json/{}.json'.format(yesterday),'r') as f:
    i = f.read()
    json = json.loads(i)
    f.close()
    
client = MongoClient() 
database = client["test"]
collection = database["yesterday"]
collection.insert_many(
    json
)